## IP address and network mask basics

**A**) We look at **networking** with **private ip** (Within a network). Communication between instances with **internal ip addresses** are deemed to be more safe. This is also called **RFC1918 connectivity space**. The data does not need to traverse the public internet, where it is succesptible for hacking.

> ### IPv4

> IP address is **32 bits** long.

>>1) **1 bit** represents **0/1**. Hence an IPv4 is a **sequence of 32 binary numbers**

>>2) **8 bits** can represent decimal numbers from **0** to **255** (2^8 = 256)

>>3) Hence **{8 bits}.{8 bits}.{8 bits}.{8 bits}** is represented from range **0.0.0.0** to **255.255.255.255**

> ### CIDR notation and network mask

> This is used to denote **IP address ranges**. 

> For IP address range assignment to 255 hosts, we can **fix** first **24** bits and **allow ranges** for **last 8 bits**. 

>Eg, Representing **192.168.1.[0 - 255]**, by fixing **first 24 bits** in **CIDR** notation is,

>> **192.168.1.0/24**

>> Address:   192.168.1.0     ==      **11000000.10101000.00000001** .00000000

>> Netmask:   255.255.255.0 = 24 ==   **11111111.11111111.11111111** .00000000

>> **HostMin**:   192.168.1.1  ==        11000000.10101000.00000001 .**00000001**

>> **HostMax**:   192.168.1.254    ==     11000000.10101000.00000001 .**11111110**

> Denoting IP address range to allow **2^23** hosts. For this, we fix the first **32-23 = (9** bits)

>> **192.0.0.0/9**

>> Address:   192.0.0.0     ==      **11000000.0** 0000000.00000000.00000000

>> Netmask:   255.128.0.0 = 9   ==    **11111111.1** 0000000.00000000.00000000

>> **HostMin**:   192.0.0.1     ==        11000000.0 **0000000.00000000.00000001**

>> **HostMax**:   192.127.255.254   ==   11000000.0 **1111111.11111111.11111110**

> The **first few bits** acts as the **ID** of the network. (**eg 192.0**). 

> Notice that, in the second example, the Host min and max will remain same for any address in the range **192.[0-127]**, as the 1st bit in the 2nd set is 0 for numbers between 0-127, after which it changes to 1. When we set the address as **192.128.0./9**, host min and max ranges change from 128-255

>> Address:   192.128.0.0   ==        **11000000.1** 0000000.00000000.00000000

>> Netmask:   255.128.0.0 = 9     ==  **11111111.1** 0000000.00000000.00000000

>> **HostMin** :   192.128.0.1  ==         11000000.1 **0000000.00000000.00000001**

>> **HostMax**:   192.255.255.254  ==     11000000.1 **1111111.11111111.11111110**

> Refer **[this page](http://jodies.de/ipcalc?host=192.128.0.0&mask1=9&mask2=)** for IP range calculations


> ### Virtual private cloud (VPC)

> When VM instances are created in GCP, they are assigned to the **default VPC network** with an **IP** assigned to **every region**. The following IP range covers the **private IP** of all possible instances in any region. Each region is a **subnet** 

>> **10.128.0.0/9**

>>> **asia south1** : **10.160.0.0/20**, where **160** is the **id for asia-south1** and instances within this region are assigned **private ip** in the range **10.160.0.1** to **10.160.15.254**

>>> That is, if we create an instance in **asia-south-1** the internal ip assigned to first instance is **10.160.0.1**

>>> Thus, the **region code** for any region are in the range **128 - 255**

> **Default VPC** have **default set of firewall rules** automatically created (Allow ssh,http,https,rdp,icmp etc)

> ### Creating custom VPC

>> To create custom VPC, we have to define **IP Id** for each **subnets**

>> Eg, For a custom VPC that allows instances in regions **asia-south1** and **europe-west1**, that allows 255 hosts in each region

>>> 1) Under **VPC** tab, click **create VPC**.

>>> 2) Click **add subnet** and choose region **asia-south1**. Specify **IP range : 10.0.1.0/24** (The first 24 bits can be anything)

>>> 3) Click **add subnet** and choose region **europe-west1**. Specify **IP range : 10.0.2.0/24** (The first 24 bits **shoud not overlap** with anyother ip in subnets)

>> If we **create VM instances under this new VPC**, we can create instances only in the regions **asia-south1** and **europe-west1**

>> By default, **Custom VPCs** will have **no fire wall rules set**. That is, we **cant even ssh into those instances.**, We have to **create firewall rules for following**,

>>> 1) To allow **ssh**, we have to **allow port 22**, tcp:22. 

>>> 2) To allow **http request**, we have to **allow port 80**

>>> 3) To allow **https request**, we have to **allow port 443**

>>> 4) To allow **rdp**, we have to **allow port 3389**

>>> 5) To allow **icmp protocol**, create firewall rule and set **other protocol : icmp**. This will allow us to use **ping** to test connection.

>> A) If we want to **allow incomming connection from any source ip** set **source ip : 0.0.0./0**

>> B) If you want to allow certain firewall rule **only from instances within VPC** add **10.0.1.0/24** and **10.0.2.0/24** to **source ip ranges**

>> C) If you want to allow **ssh/http**  connection **only from** your **local computer**, enter the **public ip** of your **local computer** as the **source ip** (google **my public ip** to find your public ip)

>> D) If you want to apply a fire wall rules just to certain instances, use **network tags**

> ### To check connection

>> **ping** can be used to check connections accross instances over the **same VPC** (includes different subnets)

>>```bash
ping [INSTANCE_NAME] (or)
ping [INTERNAL_IP]
```
>> NOTE: **ping** can also be used to check public connections. eg,

>>```bash
ping google.com
```

>> We can also use **ssh** in the following manner to connect between instances from **within network**

>>```bash
ssh -a [INSTANCE_NAME]
```

>> To ssh from local computer,
```bash
gcloud compute ssh [USERNAME]@[INSTANCE_NAME]
```

### Egress charges

> GCP charges for data set from VM instances to the internet. Egress chagres are not applied when the communication is within the same zone using **internal ip**. NOTE: If **external ips** are used, egress charges are applied **even if the instances are in the same zone**.

> Sometimes, we have VPC netwroks in different projects in same zone that needs to communicate. To enable VPC networks from different projects to communicate with **internal ip**, we should set up **network peering** between the two VPC networks from both the projects.(Internal Ips should not clash) We can also use **shared VPC** (create a Single VPC that can be shared by two projects) if the account has an **organization admin** 

To connect **GCP** with **on premise machines** use **Cloud VPN** with **cloud router**